In [296]:
import numpy as np
import pandas as pd
from glob import glob

import pandas_ta as pta
import sys
sys.path.append("..") 
from tp_config import *
from tp_utils.tp_utils import returns
from tp_utils.data_provider import read_prices, read_data,load_data_from_exchange
from tp_utils.backtest import simulation
pd.set_option('precision', 9)
np.set_printoptions(edgeitems=30, linewidth=1000, formatter=dict(float=lambda x: "%.3g" % x))

In [297]:
df_all = read_data("BTC-USDT", "3m")
df_prices = df_all[-10000:].copy()

In [298]:
log_rets = returns(df_prices['C'])
df_prices['high'] = df_prices['H']
df_prices['low'] = df_prices['L']
df_prices['close'] =  df_prices['C']

In [299]:
from statsmodels.tsa.stattools import adfuller

X = log_rets
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -15.493582
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567


In [329]:
m = 1
n = 3
k = 2
Basic= pta.Strategy(
    name="EMAs, BBs, and MACD",
    description="Non Multiprocessing Strategy by rename Columns",
    ta=[
        {"kind": "ema", "length": 8},
        {"kind": "ema", "length": 21},
#        {"kind": "bbands", "length": 20, "col_names": ("BBL", "BBM", "BBU")},
        {"kind": "macd", "fast": m, "slow": n, "signal": k, "col_names": ("MACD", "MACDh", "MACDs")},
#        {"kind": "stc"}
    ]
)
# Run it
#df.ta.strategy(NonMPStrategy)

In [326]:
df_feat = df_prices[['close']].copy()
df_feat.ta.strategy(Basic)

df_feat['p_shift'] = df_feat['close'].shift(n+k-2)
df_feat['X4'] = df_feat['MACDh']/df_feat['p_shift']

In [313]:
a = -0.00019
xx = (df_feat['X4'] < a).astype(int)
df_feat['buy'] = xx


b = 0.00023 * 1.2
xx = (df_feat['X4'] > b).astype(int)
df_feat['sell'] = xx 

df_feat['signals'] = df_feat['buy'] - df_feat['sell']

In [334]:
a = -0.00019
xx = (df_feat['X4'] < a).astype(int)
xx = xx.diff()
xx.iloc[0] = 0
df_feat['buy'] = -xx.astype(int)
df_feat['buy'] = df_feat['buy'].apply(lambda x: max(x,0))


b = 0.00023 * 1.2
b = 0
xx = (df_feat['X4'] > b).astype(int)
xx = xx.diff()
xx.iloc[0] = 0
df_feat['sell'] = -xx.astype(int)
df_feat['sell'] = df_feat['sell'].apply(lambda x: max(x,0))

df_feat['signals'] = df_feat['buy'] - df_feat['sell']

In [335]:
df_signals = df_feat[['close', 'signals']]
df_signals.to_csv('../r_macd.csv', index = False)